### Задание
Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.
Получившийся отчёт сохранить в формате Apache Parquet.

Набор данных:
- выборка данных posts_sample.xml (из stackoverflow.com-Posts.7z),
- файл со списком языков programming-languages.csv, собранных с вики-страницы

In [1]:
import findspark
findspark.init()

In [2]:
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, DateType

In [3]:
from pyspark.sql import SparkSession
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'

# Создаем объект SparkSession,точка входа для взаимодействия с оболочкой Spark
spark_session = SparkSession\
    .builder\
    .getOrCreate()

In [4]:
spark_session

In [7]:
# Создаем SparkContext, который устанавливает взаимодействие кластера и resource manager (yarn)
sc = spark_session._sc

#### Перемещаем файлы в hdfs

In [ ]:
!hadoop fs -put programming-languages.csv data/programming-languages.csv

In [ ]:
!hadoop fs -put posts_sample.xml data/posts_sample.xml

#### Считываем данные из файлов

In [8]:
# Создаем схему, определяюшие колонки для DataFrame
proglang_schema = StructType([StructField("name", StringType(), False),
    StructField("wikipedia_url", StringType(), False)]) 

In [9]:
# Считываем данные из файла в DataFrame
proglang_df = spark_session.read.csv("data/programming-languages.csv", schema=proglang_schema)

In [10]:
# Show first rows from DataFrame
proglang_df.head(3)

[Row(name='name', wikipedia_url='wikipedia_url'),
 Row(name='A# .NET', wikipedia_url='https://en.wikipedia.org/wiki/A_Sharp_(.NET)'),
 Row(name='A# (Axiom)', wikipedia_url='https://en.wikipedia.org/wiki/A_Sharp_(Axiom)')]

In [11]:
# Создаем list с названиями языков
proglang_names = [col['name'] for col in proglang_df.collect()]
proglang_names.pop(0)

'name'

In [12]:
proglang_names[:3]

['A# .NET', 'A# (Axiom)', 'A-0 System']

In [13]:
# Считываем данные из файла в DataFrame
posts_sample_df = spark_session.read.format("xml").options(rowTag="row").load('data/posts_sample.xml')

In [14]:
# Show first rows from DataFrame
posts_sample_df.head(1)

[Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 20, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 8, 1, 2, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 5, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 5, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUser

Названия языков в posts_samples указываются в колонке Tags. Чтобы определить популярность языка, посчитаем количество постов с соответсвующим ему тэгом. Год создания поста указан в теге(колонке) CreationDate.

#### Находим список топ-10 языков для каждого года

In [15]:
def check_date(row, year):
    fromdate = datetime(year=year, month=1, day=1)
    tilldate = datetime(year=year, month=12, day=31)
    created_date = row._CreationDate
    return created_date >= fromdate and created_date <= tilldate

In [16]:
def check_language(row):
    lang_tag = None
    for lang in proglang_names:
        if '<' + lang.lower() + '>' in row._Tags.lower():
            lang_tag = lang
            break
    if lang_tag is None:
        return None
    return (row._Id, lang_tag)

In [21]:
top_languages_dict = {}
years = range(2010, 2020)
for year in years:
    '''
    Choose rows with certain year that have some data in Tag,
    map it to (id of post, language),
    remove rows where language wasn't mentioned,
    change key -> (language, id of post),
    (language, id of post) -> (language, number of posts with these language) 
        within single partiotion, then it's merged into one across partitions,
    languages are sorted by number of posts in descending order,
    convert it to DataFrame
    '''
    top_langs = posts_sample_df.rdd\
        .filter(lambda row: row._Tags is not None and check_date(row, year))\
        .map(check_language)\
        .filter(lambda row: row is not None)\
        .keyBy(lambda row: row[1])\
        .aggregateByKey(
            0,
            lambda acc, value: acc + 1,
            lambda acc1, acc2: acc1 + acc2,
        )\
        .sortBy(lambda row: row[1], ascending=False)\
        .toDF()
    top_langs = top_langs.select(col('_1').alias(f'Top-10_languages'))
    top_languages_dict[year] = top_langs.limit(10)

#### Записываем таблицы в файлы parquet

In [22]:
for year in top_languages_dict.keys():
    top_languages_dict[year].write.format("parquet").save(f"output/top_programming_languages_{year}")

#### Считываем файлы parquet, чтобы вывести результаты

In [23]:
for year in range(2010, 2020):
    print(year)
    df = spark_session.read.format("parquet").load(f"output/top_programming_languages_{year}")
    df.show()

2010
+----------------+
|Top-10_languages|
+----------------+
|            Java|
|      JavaScript|
|             PHP|
|          Python|
|     Objective-C|
|               C|
|            Ruby|
|          Delphi|
|     AppleScript|
|               R|
+----------------+

2011
+----------------+
|Top-10_languages|
+----------------+
|             PHP|
|            Java|
|      JavaScript|
|          Python|
|     Objective-C|
|               C|
|            Ruby|
|            Perl|
|          Delphi|
|            Bash|
+----------------+

2012
+----------------+
|Top-10_languages|
+----------------+
|             PHP|
|      JavaScript|
|            Java|
|          Python|
|     Objective-C|
|               C|
|            Ruby|
|            Bash|
|               R|
|          MATLAB|
+----------------+

2013
+----------------+
|Top-10_languages|
+----------------+
|      JavaScript|
|            Java|
|             PHP|
|          Python|
|     Objective-C|
|               C|
|       